In [12]:
from xgboost import XGBClassifier

from sklearn.svm import SVC
from src.model.porpoise import Encoder, Model
from src.features.encodings import multiple, binary, ncp, pstnpss, pse_knc
from src.dataset import load_benchmark_dataset, Species, Modification

In [2]:
#encoder = multiple.Encoder(encoders=[
#     binary.Encoder(),
#     ncp.Encoder(),
#     pstnpss.Encoder(),
#     pse_knc.Encoder()
# ])

encoder = Encoder(species=Species.human)

In [3]:
train_dataset = load_benchmark_dataset(Species.human, Modification.psi)
test_dataset = load_benchmark_dataset(Species.human, Modification.psi, True)

In [4]:
train_y = train_dataset.targets
train_x = encoder.fit_transform(train_dataset, y=train_y).samples
# 
test_y = test_dataset.targets
test_x = encoder.transform(test_dataset).samples

In [15]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
# from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Function to cluster data and classify each cluster
def cluster_and_classify(train_x, train_y, test_x, test_y, n_clusters=3, clustering_algorithm='kmeans', classifier=None):
    # if clustering_algorithm == 'kmeans':
    # cluster_model = GaussianMixture(n_components=n_clusters, random_state=42)
    cluster_model = GaussianMixture(n_components=n_clusters, random_state=42)
    # elif clustering_algorithm == 'gmm':
    #     cluster_model = GaussianMixture(n_components=n_clusters, random_state=42)
    # else:
    #     raise ValueError("Unsupported clustering algorithm. Use 'kmeans' or 'gmm'.")

    clusters_train = cluster_model.fit_predict(train_x)
    clusters_test = cluster_model.predict(test_x)
    
    if classifier is None:
        classifier = SVC(kernel='linear')

    cluster_classifiers = []
    for cluster_id in range(n_clusters):
        cluster_train_x = train_x[clusters_train == cluster_id]
        cluster_train_y = train_y[clusters_train == cluster_id]

        if len(cluster_train_y) == 0:
            print(f"No data for cluster {cluster_id}. Skipping.")
            cluster_classifiers.append(None)
            continue

        clf = classifier.fit(cluster_train_x, cluster_train_y)
        cluster_classifiers.append(clf)

    for cluster_id in range(n_clusters):
        cluster_test_x = test_x[clusters_test == cluster_id]
        cluster_test_y = test_y[clusters_test == cluster_id]

        if cluster_classifiers[cluster_id] is None or len(cluster_test_y) == 0:
            print(cluster_classifiers[cluster_id])
            print(f"No data for cluster {cluster_id}. Skipping.")
            continue

        predictions = cluster_classifiers[cluster_id].predict(cluster_test_x)
        print(f"Classification report for cluster {cluster_id}:")
        print(classification_report(cluster_test_y, predictions))

# Example usage:
# Assuming train_x, train_y, test_x, and test_y are already defined and loaded

# Define the number of clusters and the clustering algorithm
n_clusters = 3
clustering_algorithm = 'kmeans'  # or 'gmm'

# Call the function
cluster_and_classify(train_x, train_y, test_x, test_y, n_clusters=n_clusters, clustering_algorithm=clustering_algorithm)


Classification report for cluster 0:
              precision    recall  f1-score   support

           0       0.82      0.38      0.51        24
           1       0.38      0.82      0.51        11

    accuracy                           0.51        35
   macro avg       0.60      0.60      0.51        35
weighted avg       0.68      0.51      0.51        35

Classification report for cluster 1:
              precision    recall  f1-score   support

           0       0.50      0.33      0.40        49
           1       0.65      0.79      0.72        78

    accuracy                           0.61       127
   macro avg       0.58      0.56      0.56       127
weighted avg       0.59      0.61      0.59       127

Classification report for cluster 2:
              precision    recall  f1-score   support

           0       0.82      0.67      0.73        27
           1       0.44      0.64      0.52        11

    accuracy                           0.66        38
   macro avg     